# Assignment 2: Scalarizing functions

Different scalarizing functions will produce different solutions with the same preference information.
Use different scalarizing functions to compute different solutions with the same preference information.

Report on your results and comment on the solutios you have got.

## Hints

- An example below has been provided to show how a scalarized problem can be solved.
- Different scalarization functions can be found in the modules found in `desdeo_tools.scalarization`.
    - Some of the available scalarizing functions are documented [here](https://desdeo-tools.readthedocs.io/en/latest/autoapi/scalarization/ASF/index.html). These need to be setup before utilizing them to setup an instance of `Scalarizer`!
- The logic to build is a scalarized problem is the following (also shown in the example code below):
    1. First, we need a problem and a scalarization function.
    2. Then, we need to scalarize the problem utilizing a `Scalarizer`.
    3. Lastly, we need a solver (`ScalarMinimizer`) to tolve the scalarized problem.
- Some scalarizing functions will require information about the ideal or the nadir point, or both. Use the values you got in the first assigment in this excercise set.

In [2]:
# Example on how to scalarize and solve a scalarized problem

import numpy as np

from desdeo_problem.problem import MOProblem
from desdeo_problem.testproblems import test_problem_builder 
from desdeo_tools.solver import ScalarMinimizer
from desdeo_tools.scalarization import Scalarizer

# Dummy scalarizing functions, do not use this, it is bad! (why?)
def dummy_scalarizer(objective_vector: np.ndarray) -> float:
    ys = np.atleast_2d(objective_vector)
    
    # you can try to change these weights and see how they affect the solution
    ws = np.array([0.2, 0.4, 0.4])
    
    return np.sum(objective_vector * ws, axis=1)

# DTLZ2 as an example problem to solve
dtlz2 = test_problem_builder("DTLZ2", n_of_variables=5, n_of_objectives=3)

# Scalarize the problem
sum_scalarazer = Scalarizer(lambda x: dtlz2.evaluate(x).objectives, dummy_scalarizer)

# Setup a solver
minimizer = ScalarMinimizer(sum_scalarazer, dtlz2.get_variable_bounds(), method="scipy_minimize")

# Solve
res = minimizer.minimize(np.array(5*[0.5]))

xs, fs = res.x, dtlz2.evaluate(res.x).objectives

print(f"Solution:\nDecision variables: {xs}\nObjective vector: {fs}")

Solution:
Decision variables: [1.e-06 1.e-06 5.e-01 5.e-01 5.e-01]
Objective vector: [[1.00000000e+00 1.57079633e-06 1.57079633e-06]]


In [3]:
from desdeo_tools.scalarization.ASF import StomASF

In [27]:
ideal_point = np.array([0, 0, 0])
stom_fun = StomASF(ideal_point)

reference_point = np.array([0.2, 0.3, 0.1])
objective_function = lambda x: dtlz2.evaluate(x).objectives

stom_scalarizer = Scalarizer(objective_function, stom_fun, scalarizer_args={"reference_point": reference_point})

stom_minimizer = ScalarMinimizer(stom_scalarizer, dtlz2.get_variable_bounds(), method="scipy_de") # scipy_minimize

In [21]:
res_stom = stom_minimizer.minimize(np.array(5*[0.5]))

In [22]:
res_stom

     fun: 2.678513780047446
     jac: array([-1.16690786e+00,  1.07378355e+00, -9.69935244e-04, -1.55893076e-01,
       -1.96910532e-01])
 message: 'Optimization terminated successfully.'
    nfev: 1893
     nit: 16
 success: True
       x: array([0.17223784, 0.62566561, 0.49981853, 0.47083501, 0.46316135])

In [24]:
print(f"Decison variables: {res_stom.x}; objective functions values {dtlz2.evaluate(res_stom.x).objectives}; reference point; {reference_point}")

Decison variables: [0.17223784 0.62566561 0.49981853 0.47083501 0.46316135]; objective functions values [[0.53570283 0.8035534  0.26785206]]; reference point; [0.2 0.3 0.1]
